In [172]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sqlite3

con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/dCleaned.db')

In [0]:
data = []
with con:    
    cur = con.cursor()    
    cur.execute("SELECT * FROM districts")
    rows = cur.fetchall()
 
    for row in rows:
        data.append(row)

In [175]:
data[0]

(550,
 'автозавод',
 'Если кто-то найдет сережку, верните, пожалуйста, очень дорога как память 😭',
 'найдет вернуть пожалуйста очень дорога память',
 1,
 0,
 0,
 0,
 0,
 3735,
 0,
 7,
 2,
 9,
 '2019-08-30')

In [176]:

! pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
import re

In [0]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [0]:
from gensim.models import CoherenceModel

In [179]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [180]:
! pip install pymorphy2
import pymorphy2 
morph = pymorphy2.MorphAnalyzer()

In [0]:
import gensim

In [182]:
morph.parse('сережка')[0].normal_form

'серёжка'

In [0]:
'''
все районы
'''

cur_data = {}
for i in data:
    ind = i[0]
    sent = i[2].lower()
    tokens = [morph.parse(tok)[0].normal_form for tok in tokenizer.tokenize(sent) if tok.isalpha() and len(tok) > 2 and tok not in stop_words
              and bool(re.match(r'[а-я]', tok)) and morph.parse(tok)[0].tag.POS not in ['NPRO', "NUMR", "CONJ", "INTJ", "PRCL", "PREP"]]
    cur_data[ind] = tokens  

In [0]:
"""
Построить отдельно по данным каждого района. CUR_DATA на все данные, тут - на каждый район. Далее для объединения в 
биграммы надо будет менять переменную

"""

def singleout(name):
    district = {}
    for i in data:
        if i[1] == name:
            ind = i[0]
            sent = i[2].lower()
            tokens = [morph.parse(tok)[0].normal_form for tok in tokenizer.tokenize(sent) if tok.isalpha() and len(tok) > 2 and tok not in stop_words
                    and bool(re.match(r'[а-я]', tok)) and morph.parse(tok)[0].tag.POS not in ['NPRO', "NUMR", "CONJ", "INTJ", "PRCL", "PREP"]]
            district[ind] = tokens 
    return district

avtaz = singleout('автозавод')
priok = singleout('приокский')
sor = singleout('сормово')

In [0]:
#cur_data = sor

In [0]:
data_words = list(cur_data.values())

In [0]:
bigram = gensim.models.Phrases(data_words, min_count=100, threshold=100)

In [0]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [0]:

composed = {}
for k, v in cur_data.items():
    composed[k] = bigram_mod[v]

In [0]:
id2word = gensim.corpora.Dictionary(composed.values())

In [0]:

corpus = [id2word.doc2bow(text) for text in composed.values()]

In [0]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=42,
                                           update_every=1,
                                           chunksize=128, 
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [193]:
lda_model.print_topics()

[(0,
  '0.011*"обучение" + 0.011*"рубль" + 0.010*"квартира" + 0.009*"магазин" + 0.008*"график" + 0.008*"вознаграждение" + 0.008*"снять" + 0.007*"наш" + 0.006*"вчера" + 0.006*"водитель"'),
 (1,
  '0.024*"щербинка" + 0.013*"работа" + 0.012*"цена" + 0.012*"сормово" + 0.008*"требоваться" + 0.008*"гагарин" + 0.006*"купить" + 0.005*"самый" + 0.004*"завод" + 0.004*"заказ"'),
 (2,
  '0.019*"найти" + 0.016*"район" + 0.015*"весь" + 0.009*"день" + 0.009*"писать" + 0.008*"улица" + 0.008*"год" + 0.007*"мочь" + 0.006*"человек" + 0.006*"телефон"'),
 (3,
  '0.021*"год" + 0.009*"группа" + 0.008*"который" + 0.006*"нижний_новгород" + 0.006*"пост" + 0.006*"класс" + 0.006*"город" + 0.005*"наш" + 0.005*"дело" + 0.005*"подарок"'),
 (4,
  '0.020*"дом" + 0.018*"очень" + 0.010*"девочка" + 0.010*"месяц" + 0.009*"добрый" + 0.008*"рука" + 0.008*"человек" + 0.008*"искать" + 0.008*"звонить" + 0.007*"хозяин"')]

In [165]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.255374  0.006310       1        1  34.318901
4     -0.161277  0.076613       2        1  25.633282
3      0.063866 -0.244400       3        1  19.087852
1      0.180863  0.202959       4        1  10.949044
0      0.171922 -0.041482       5        1  10.010922, topic_info=     Category        Freq       Term       Total  loglift  logprob
2     Default  640.000000      найти  640.000000  30.0000  30.0000
4570  Default  263.000000   щербинка  263.000000  29.0000  29.0000
96    Default  527.000000      район  527.000000  28.0000  28.0000
94    Default  665.000000        дом  665.000000  27.0000  27.0000
3     Default  591.000000      очень  591.000000  26.0000  26.0000
...       ...         ...        ...         ...      ...      ...
519    Topic5   60.009750      школа  115.272774   1.6487  -5.1036
347    Topic5   68.342331        наш  283.890869   0.8774  -4.9736
513    Topic5   42.998333      центр   85.571236   1.6133  -5.4370
1807   Topic5   36.673244  приходить   52.526482   1.9422  -5.5961
417    Topic5   42.184364     работа  288.878754   0.3775  -5.4561

[253 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2422      4  0.986218  автомобиль
7564      5  0.975712   автошкола
210       1  0.990375       админ
799       1  0.991353       адрес
1541      4  0.975029       айфон
...     ...       ...         ...
519       3  0.477129       школа
519       5  0.520505       школа
943       2  0.997005       щенок
4570      4  0.998274    щербинка
2046      4  0.983150        этаж

[291 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 2, 1])

In [0]:
perplexity = lda_model.log_perplexity(corpus)

In [167]:
perplexity

-8.423695744282794

In [0]:
coherence_model_lda = CoherenceModel(model=lda_model, texts= composed.values(), dictionary=id2word, coherence='c_v')

In [0]:
coherence_lda = coherence_model_lda.get_coherence()

In [171]:
coherence_lda

0.37855030056644534

# LDAmallet

In [206]:
 ! wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

--2020-02-17 16:54:40--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  4.65MB/s    in 3.3s    

2020-02-17 16:54:44 (4.65 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]



In [0]:
! unzip mallet-2.0.8.zip

In [0]:
mallet_path = 'mallet-2.0.8/bin/mallet' 

In [229]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=5, id2word=id2word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [230]:
ldamallet.show_topics(formatted=False)

[(0,
  [('найти', 0.04840440553515956),
   ('район', 0.0225360067777464),
   ('улица', 0.021914713357808528),
   ('просьба', 0.013160124258683988),
   ('видеть', 0.012143462298785653),
   ('телефон', 0.011578650098842135),
   ('знать', 0.011522168878847784),
   ('сегодня', 0.009714769839028524),
   ('потерять', 0.009714769839028524),
   ('имя', 0.009545326179045467)]),
 (1,
  [('помощь', 0.015399830460582086),
   ('ребёнок', 0.015352736177828012),
   ('наш', 0.014410850522746538),
   ('рубль', 0.012950927757370256),
   ('группа', 0.00993689366110954),
   ('помочь', 0.00979561081284732),
   ('ваш', 0.0087595365922577),
   ('карта', 0.008712442309503627),
   ('год', 0.008429876612979184),
   ('друг', 0.008194405199208816)]),
 (2,
  [('год', 0.029700552000390797),
   ('который', 0.01362903619754775),
   ('один', 0.011284255776464266),
   ('нижний_новгород', 0.01089345903961702),
   ('сормово', 0.010698060671193395),
   ('место', 0.009134873723804407),
   ('нижегородский', 0.00903717453959

In [0]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=composed.values(), dictionary=id2word, coherence='c_v')

In [0]:
coherence_ldamallet = coherence_model_ldamallet.get_coherence()

In [226]:
coherence_ldamallet

0.44070001030870587

In [236]:
import pandas as pd
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=list(composed.values()))

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.0,0.2440,"найти, район, улица, просьба, видеть, телефон,...","[найти, серёжка, вернуть, очень, дорога, память]"
1,1,0.0,0.2528,"найти, район, улица, просьба, видеть, телефон,...","[искать, девушка, который, ехать, маршрутка, с..."
2,2,4.0,0.3376,"очень, дом, искать, добрый, собака, хозяин, че...","[ребята, распространить, запись, отдать, добры..."
3,3,0.0,0.2561,"найти, район, улица, просьба, видеть, телефон,...","[найти, карта, имя, сергей, колебакина, автобу..."
4,4,4.0,0.2769,"очень, дом, искать, добрый, собака, хозяин, че...","[чёрный, красавчик, прибиться, космический, чи..."
5,5,0.0,0.2113,"найти, район, улица, просьба, видеть, телефон,...",[микрорайон]
6,6,0.0,0.2542,"найти, район, улица, просьба, видеть, телефон,...","[пропасть, бенгальский, кошка, улица, лесков, ..."
7,7,4.0,0.3427,"очень, дом, искать, добрый, собака, хозяин, че...","[каждый, малыш, искать, свой, хозяин, малыш, м..."
8,8,4.0,0.2264,"очень, дом, искать, добрый, собака, хозяин, че...","[красивый, одинокий, девушка]"
9,9,0.0,0.2437,"найти, район, улица, просьба, видеть, телефон,...","[искать, котик, гулять, район, дом, южный, бул..."
